In [ ]:
from google.colab import files
uploaded = files.upload()

Saving SAMPLE.csv to SAMPLE.csv
Saving TEST_FINAL.csv to TEST_FINAL.csv
Saving TRAIN.csv to TRAIN.csv


In [ ]:
import pandas as pd

train_df = pd.read_csv("TRAIN.csv", low_memory=False)
test_df = pd.read_csv("TEST_FINAL.csv")


In [ ]:
def split_date(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['WeekOfYear_x'] = df.Date.dt.isocalendar().week

In [ ]:
split_date(train_df)
split_date(test_df)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ID             188340 non-null  object        
 1   Store_id       188340 non-null  int64         
 2   Store_Type     188340 non-null  object        
 3   Location_Type  188340 non-null  object        
 4   Region_Code    188340 non-null  object        
 5   Date           188340 non-null  datetime64[ns]
 6   Holiday        188340 non-null  int64         
 7   Discount       188340 non-null  object        
 8   #Order         188340 non-null  int64         
 9   Sales          188340 non-null  float64       
 10  Year           188340 non-null  int64         
 11  Month          188340 non-null  int64         
 12  Day            188340 non-null  int64         
 13  WeekOfYear_x   188340 non-null  UInt32        
dtypes: UInt32(1), datetime64[ns](1), float64(1), int64(6

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22265 entries, 0 to 22264
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             22265 non-null  object        
 1   Store_id       22265 non-null  int64         
 2   Store_Type     22265 non-null  object        
 3   Location_Type  22265 non-null  object        
 4   Region_Code    22265 non-null  object        
 5   Date           22265 non-null  datetime64[ns]
 6   Holiday        22265 non-null  int64         
 7   Discount       22265 non-null  object        
 8   Year           22265 non-null  int64         
 9   Month          22265 non-null  int64         
 10  Day            22265 non-null  int64         
 11  WeekOfYear_x   22265 non-null  UInt32        
dtypes: UInt32(1), datetime64[ns](1), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
input_cols=['Store_id', 'Store_Type', 'Location_Type', 'Region_Code', 'Holiday', 
              'Discount', 
              'Day', 'Month', 'Year', 'WeekOfYear_x']
target_col=['Sales']

In [ ]:
input_x=train_df[input_cols].copy()
target_x=train_df[target_col].copy()

In [ ]:
input_test=test_df[input_cols].copy()

In [ ]:
numeric_cols = ['Store_id',
              'Day', 'Month', 'Year', 'WeekOfYear_x',  ]
categorical_cols = ['Store_Type', 'Region_Code', 'Location_Type', 'Discount']

In [ ]:
input_x[numeric_cols].isna().sum()

Store_id        0
Day             0
Month           0
Year            0
WeekOfYear_x    0
dtype: int64

In [ ]:
input_test[numeric_cols].isna().sum()

Store_id        0
Day             0
Month           0
Year            0
WeekOfYear_x    0
dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(input_x[numeric_cols])

In [ ]:
input_x[numeric_cols] = scaler.transform(input_x[numeric_cols])
input_test[numeric_cols] = scaler.transform(input_test[numeric_cols])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(input_x[categorical_cols])
encoded_cols = list(encoder.get_feature_names(categorical_cols))

input_x[encoded_cols] = encoder.transform(input_x[categorical_cols])
input_test[encoded_cols] = encoder.transform(input_test[categorical_cols])

In [ ]:
X = input_x[numeric_cols + encoded_cols]
X_test = input_test[numeric_cols + encoded_cols]

In [ ]:
X_test

,Store_id,Day,Month,Year,WeekOfYear_x,Store_Type_S1,Store_Type_S2,Store_Type_S3,Store_Type_S4,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Discount_No,Discount_Yes
0,0.467033,0.0,0.454545,1.0,0.411765,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.469780,0.0,0.454545,1.0,0.411765,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.472527,0.0,0.454545,1.0,0.411765,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.475275,0.0,0.454545,1.0,0.411765,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.464286,0.0,0.454545,1.0,0.411765,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22260,0.508242,1.0,0.545455,1.0,0.588235,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
22261,0.027473,1.0,0.545455,1.0,0.588235,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
22262,0.505495,1.0,0.545455,1.0,0.588235,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
22263,0.186813,1.0,0.545455,1.0,0.588235,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
from sklearn.linear_model import LassoLars

las = LassoLars(alpha=0.3, fit_intercept=False, normalize=True)
lasreg = las.fit(X, target_x)

In [ ]:
pred=lasreg.predict(X)
pred

array([41819.8727884 , 70486.78920523, 61586.43439941, ...,
       29021.07477367, 50185.64965475, 29408.04756056])

In [ ]:
test_preds = lasreg.predict(X_test)
test_preds

array([58297.83466173, 35113.82948522, 59197.99667517, ...,
       45040.78871171, 32833.69440293, 28581.25925811])

In [ ]:
submission_df = pd.read_csv("SAMPLE.csv")

In [ ]:
submission_df['Sales']  = test_preds

In [ ]:
submission_df

,ID,Sales
0,T1188341,58297.834662
1,T1188342,35113.829485
2,T1188343,59197.996675
3,T1188344,33980.235887
4,T1188345,34487.397071
...,...,...
22260,T1210601,18396.609827
22261,T1210602,58045.024716
22262,T1210603,45040.788712
22263,T1210604,32833.694403


In [ ]:
from google.colab import files

submission_df.to_csv('submission_Bayesian_Ridge_Regression.csv', index=False)
files.download('submission_Bayesian_Ridge_Regression.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>